# [Mars Lander](https://www.codingame.com/ide/puzzle/mars-lander)

My Ynote: "Coding Games - Mars Lander"<br>
[Study "Genetic Algorithm Implementation in Python" with helps of peforth](https://github.com/hcchengithub/GeneticAlgorithmPython/blob/master/Genetic%20Algorithm%20Implementation%20in%20Python.ipynb)

I didn't understand this game at first and that's a big concern to invest my time into developing the program. Thanks to [McKelian](https://www.codingame.com/forum/t/mars-the-game/756) and his [video game](http://www.mckelian.com/index.php/mars/choix) that mimics the Mars Lander puzzle.

<img src=marslander.png>
<img src=ControlPanel.png>

### Goal 
- land on flat ground
- land in a vertical position (tilt angle = 0°)
- vertical speed must be limited ( ≤ 40m/s in absolute value)
- horizontal speed must be limited ( ≤ 20m/s in absolute value)


# 調製 Simulator 成功了！

這套公式成功了。

Initialize

    thrust = 4    # the thrust command 
    rotate = 0    # the rotate command 
    a0 = 0        # absolute angle of the ship 
    pwr0 = 0      # 引擎出力
    gravity = 3.711  # constant on Mars
    p0 = v(2500, 2700)  # Initial position
    s0 = v(0,0)     # Initial speed vector
    g = angle2uv(-180) * gravity  # 重力向量
    step = 0
    
Run

    # f = m.a  因為 m 不變故當成 1 所以 f = a 也就是 force = g 
    # Duration t = 1 一步的時間都是 1 所以移動距離 s0 + (1/2)(force)(t^2) 變成 s0 + (1/2)*(force)
    
    step += 1

    a1 = angle15(a0,rotate)  # 這一步的角度 
    pwr1 = thrust_1(pwr0,thrust)  # 這一步的 引擎推力
    force = g + angle2uv(a1) * pwr1   # force on the ship during this step 
    s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration) 
                     # where delta time is 1, a is force as mentioned above.
    p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置

    %f step -->
    %f a0 pwr0 -2 slice -->  # 上一步的 角度 與 引擎出力 0 1 2 3 4
    %f a1 pwr1 -2 slice -->  # 作用在這一步上的 角度 與 引擎出力
    %f force dup -->
    %f ===>      # 重力 與 引擎出力 的合力向量
    %f s1 ===>   # 這一步最後的速度
    %f p1 ===>   # 這一步最後的位置

    a0 = a1
    pwr0 = pwr1
    s0 = s1
    p0 = p1

# 寫出評分 function 

- 到達地表時的 位置、速度、角度、油料 決定得分。


# Laboratory

In [1]:
import numpy as np
import re, math, peforth, random, pdb, json 
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

reDef unknown
reDef -->
p e f o r t h    v1.24
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



### Read log file 

In [40]:
# Method 1, from another notebook
%store -r log
log

[{'step': 0,
  'length': 91,
  'x': 2500,
  'y': 2500,
  'hspeed': 0,
  'vspeed': 0,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (-99, -99)},
 {'step': 1,
  'length': 91,
  'x': 2500,
  'y': 2498,
  'hspeed': 0,
  'vspeed': -4,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 2,
  'length': 91,
  'x': 2500,
  'y': 2493,
  'hspeed': 0,
  'vspeed': -7,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 3,
  'length': 91,
  'x': 2500,
  'y': 2483,
  'hspeed': 0,
  'vspeed': -11,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 4,
  'length': 91,
  'x': 2500,
  'y': 2470,
  'hspeed': 0,
  'vspeed': -15,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 5,
  'length': 91,
  'x': 2500,
  'y': 2454,
  'hspeed': 0,
  'vspeed': -19,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 6,
  'length': 91,
  'x': 2500,
  'y': 2433,
  'hspeed': 0,
  'vspeed': -22,
  'fuel': 500,
  'angle':

In [146]:
# Method 2, Read back the json file
%f s" actual free fall.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()

### Free Fall simulatin is ok 

In [672]:
# Run once for initial state 
gravity = -3.711        # constant on Mars
g = gravity            # 重力轉成向量
step = 0 
gene = log[step]
p0 = gene['y']
s0 = gene['vspeed']
fuel0 = gene['fuel']
a0 = gene['angle']    # 這一步的角度 
powr0 = gene['power']  # 這一步的 引擎推力
cmd0 = (0,0)
state0 = (step ,p0 ,s0 ,fuel0 ,a0 ,powr0 ,cmd0)
log0 = (log[0]['step'] ,log[0]['y'] ,log[0]['vspeed'] ,log[0]['fuel'] ,log[0]['angle'] ,log[0]['power'] ,log[0]['cmd'])
%f log :> [0] -->
%f state0 -->
%f log0 -->


log :> [0] --> {'step': 0, 'length': 36, 'x': 2500, 'y': 2500, 'hspeed': 0, 'vspeed': 0, 'fuel': 500, 'angle': 0, 'power': 0, 'cmd': [-99, -99]} (<class 'dict'>)
state0 --> (0, 2500, 0, 500, 0, 0, (0, 0)) (<class 'tuple'>)
log0 --> (0, 2500, 0, 500, 0, 0, [-99, -99]) (<class 'tuple'>)


In [708]:
# run multiple times to see if the simulation match with the log, it does!!
step += 1
a1 = 0
powr1 = 0  # free fall 實驗專用
fuel1 = fuel0
force = g # + angle2uv(a1) * pwr1   # force on the ship during this step 
s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration)  # where delta time is 1, a is force as mentioned above.
p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel0 ,a0 ,powr0 ,cmd0)
p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
log1 = (step ,log[step]['y'] ,log[step]['vspeed'] ,log[step]['fuel'] ,log[step]['angle'] ,log[step]['power'] ,log[step]['cmd'])

%f ( log-length ) log :> [0]['length'] -->
%f state1 -->
%f log1 -->


( log-length ) log :> [0]['length'] --> 36 (<class 'int'>)
state1 --> (36, 95, -134, 500, 0, 0, (0, 0)) (<class 'tuple'>)
log1 --> (36, 100, -134, 500, 0, 0, [0, 0]) (<class 'tuple'>)


In [710]:
# Scoring and get the cost
%f p1 -->
%f ( 觸地了嗎？ ) p1 100 <= -->
%f s1 -->
%f ( 安全著陸嗎？ ) s1 abs 40 <= -->
%f ( 若安全著陸，所剩油料就是分數 ) fuel1 -->
%f ( 觸地時的 power ) powr0 -->

p1 --> 95.2720000000003 (<class 'float'>)
( 觸地了嗎？ ) p1 100 <= --> True (<class 'bool'>)
s1 --> -133.59599999999998 (<class 'float'>)
( 安全著陸嗎？ ) s1 abs 40 <= --> False (<class 'bool'>)
( 若安全著陸，所剩油料就是分數 ) fuel1 --> 500 (<class 'int'>)
( 觸地時的 power ) powr0 --> 0 (<class 'int'>)


In [1053]:
# 墜毀時的速度減到+-40 以內需要多少油料？ 從當時的 power 開始要用力噴火減速。
# 結果發現，修正自由落體墜毀需要的油料繼續從 fuel 裡扣除，
# 最後修正到 vspeed = -40 時 fuel = -866 此即這個 genome 的分數了。
step += 1
a1 = 0
powr1 = min(powr0 + 1, 4)
fuel1 = fuel0 - powr1
force = g + powr1
s1 = s0 + force
p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))
p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
%f state1 -->

state1 --> (379, -30779, -40, -866, 0, 4, (0, 4)) (<class 'tuple'>)


## Free fall 10 steps and then thrust with all power

In [3]:
# 這個模擬已經成功了。
# Run once for initial state 
gravity = -3.711        # constant on Mars
g = gravity            # 重力轉成向量
step = 0 
gene = log[step]
p0 = gene['y']
s0 = gene['vspeed']
fuel0 = gene['fuel']
a0 = gene['angle']    # 這一步的角度 
powr0 = gene['power']  # 這一步的 引擎推力
cmd0 = (0,0)
state0 = (step ,p0 ,s0 ,fuel0 ,a0 ,powr0 ,cmd0)
log0 = (log[0]['step'] ,log[0]['y'] ,log[0]['vspeed'] ,log[0]['fuel'] ,log[0]['angle'] ,log[0]['power'] ,log[0]['cmd'])
%f state0 -->
%f log0 -->


state0 --> (0, 2500, 0, 500, 0, 0, (0, 0)) (<class 'tuple'>)
log0 --> (0, 2500, 0, 500, 0, 0, [-99, -99]) (<class 'tuple'>)


In [94]:
# run multiple times to see if the simulation match with the log, it does!!
step += 1
a1 = 0
powr1 = 0 if step < 10 else min(powr0+1,4)  # 本實驗專用 
fuel1 = fuel0 - powr1
force = g + powr1   # force on the lander ship during this step 
s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration)  # where delta time is 1, a is force as mentioned above.
p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))

p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
try:
    log1 = (step ,log[step]['y'] ,log[step]['vspeed'] ,log[step]['fuel'] ,log[step]['angle'] ,log[step]['power'] ,log[step]['cmd'])
except:
    pass
%f state1 -->
%f log1 -->

# 結果觸地時的 vspeed 是 -16 成功著陸，分數就是剩餘油料 178 
# state1 --> (91, 98, -16, 178, 0, 4, (0, 4)) (<class 'tuple'>)
# log1 --> (91, 100, -16, 178, 0, 4, [0, 4]) (<class 'tuple'>)

state1 --> (91, 98, -16, 178, 0, 4, (0, 4)) (<class 'tuple'>)
log1 --> (91, 100, -16, 178, 0, 4, [0, 4]) (<class 'tuple'>)


## Thrust with all power 直到油料耗盡

油料耗盡時的高度距離與速度以自由落體觸地 (期間沒有耗用油料)，然後把速度調整到+-40 以內需要多少油料？這時候從空中 free fall 下來的 vspeed 一定是負的，只有 -40 以下才需要噴火調整，故只有一個方向 (不用考慮有另一方向也需要調整的可能)。

從頭到尾全速噴火直到油料耗盡的程式很簡單，只改一行：

    # 2 integers: rotate power. rotate is the desired rotation angle (should be 0 for level 1), power is the desired thrust power (0 to 4).
    print("0 4")  # 0 0 改成 0 4 

結果 the lander 飛走了 (y > 3000) 而被判 failed. Log 檔 "all power.json". 這組 log 有高度、速度、推力、油料等數據，照樣可以用來練習油料耗盡的狀況，因此時油料並未耗盡，故反而更周延。

In [1056]:
# Read the log json file
%f s" all power.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()
log

reDef log


[{'step': 0,
  'length': 83,
  'x': 2500,
  'y': 2500,
  'hspeed': 0,
  'vspeed': 0,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': [-99, -99]},
 {'step': 1,
  'length': 83,
  'x': 2500,
  'y': 2499,
  'hspeed': 0,
  'vspeed': -3,
  'fuel': 499,
  'angle': 0,
  'power': 1,
  'cmd': [0, 4]},
 {'step': 2,
  'length': 83,
  'x': 2500,
  'y': 2495,
  'hspeed': 0,
  'vspeed': -4,
  'fuel': 497,
  'angle': 0,
  'power': 2,
  'cmd': [0, 4]},
 {'step': 3,
  'length': 83,
  'x': 2500,
  'y': 2490,
  'hspeed': 0,
  'vspeed': -5,
  'fuel': 494,
  'angle': 0,
  'power': 3,
  'cmd': [0, 4]},
 {'step': 4,
  'length': 83,
  'x': 2500,
  'y': 2485,
  'hspeed': 0,
  'vspeed': -5,
  'fuel': 490,
  'angle': 0,
  'power': 4,
  'cmd': [0, 4]},
 {'step': 5,
  'length': 83,
  'x': 2500,
  'y': 2481,
  'hspeed': 0,
  'vspeed': -5,
  'fuel': 486,
  'angle': 0,
  'power': 4,
  'cmd': [0, 4]},
 {'step': 6,
  'length': 83,
  'x': 2500,
  'y': 2476,
  'hspeed': 0,
  'vspeed': -4,
  'fuel': 482,
  'angle': 0,


In [1321]:
# Run once for initial state 
gravity = -3.711        # constant on Mars
g = gravity            # 重力轉成向量
step = 0 
gene = log[step]
p0 = gene['y']
s0 = gene['vspeed']
fuel0 = gene['fuel']
a0 = gene['angle']    # 這一步的角度 
powr0 = gene['power']  # 這一步的 引擎推力
cmd0 = (0,0)
state0 = (step ,p0 ,s0 ,fuel0 ,a0 ,powr0 ,cmd0)
log0 = (log[0]['step'] ,log[0]['y'] ,log[0]['vspeed'] ,log[0]['fuel'] ,log[0]['angle'] ,log[0]['power'] ,log[0]['cmd'])

%f log :> [0] -->
%f state0 -->
%f log0 -->


log :> [0] --> {'step': 0, 'length': 83, 'x': 2500, 'y': 2500, 'hspeed': 0, 'vspeed': 0, 'fuel': 500, 'angle': 0, 'power': 0, 'cmd': [-99, -99]} (<class 'dict'>)
state0 --> (0, 2500, 0, 500, 0, 0, (0, 0)) (<class 'tuple'>)
log0 --> (0, 2500, 0, 500, 0, 0, [-99, -99]) (<class 'tuple'>)


In [1404]:
# run multiple times to see if the simulation match with the log, it does!!
step += 1
a1 = 0
powr1 = min(powr0+1,4)  # all power 實驗專用。這裡一開始搞錯了！不是直接放 4 !!!
fuel1 = fuel0 - powr1
force = g + powr1   # force on the lander ship during this step 
s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration)  # where delta time is 1, a is force as mentioned above.
p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))

p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
try:
    log1 = (step ,log[step]['y'] ,log[step]['vspeed'] ,log[step]['fuel'] ,log[step]['angle'] ,log[step]['power'] ,log[step]['cmd'])
except:
    pass
%f ( log-length ) log :> [0]['length'] -->
%f state1 -->
%f log1 -->

# 結果到了 step 83 lander 高度超過 3000 被判定為飛走了 
# ( log-length ) log :> [0]['length'] --> 83 (<class 'int'>)
# state1 --> (83, 3004, 18, 174, 0, 4, (0, 4)) (<class 'tuple'>)
# log1 --> (83, 3004, 18, 174, 0, 4, [0, 4]) (<class 'tuple'>)

( log-length ) log :> [0]['length'] --> 83 (<class 'int'>)
state1 --> (83, 3004, 18, 174, 0, 4, (0, 4)) (<class 'tuple'>)
log1 --> (83, 3004, 18, 174, 0, 4, [0, 4]) (<class 'tuple'>)


In [1459]:
# Initialize 
# 從被判 fail 的狀態開始模擬自由落體到觸地為止
step, p0, s0, fuel0, powr0 = log1[0], log1[1], log1[2], log1[3], log1[5]
state0 = (step ,p0 ,s0 ,fuel0 ,log1[4] ,powr0 )
%f log1 -->
%f state0 -->

log1 --> (83, 3004, 18, 174, 0, 4, [0, 4]) (<class 'tuple'>)
state0 --> (83, 3004, 18, 174, 0, 4) (<class 'tuple'>)


In [1506]:
# run multiple times to simulate free fall 
step += 1
a1 = 0
powr1 = max(powr0-1,0)  # 半途開始模擬 free fall 
fuel1 = fuel0 - powr1
force = g + powr1 
s1 = s0 + force 
p1 = p0 + s0 + (1/2)*(force)
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))
p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
%f state1 -->

# free fall 到 step 130 觸地，這時 vspeed -150 準備用來換算成修正所需的油料。
# state1 --> (130, 26, -150, 168, 0, 0, (0, 0)) (<class 'tuple'>)

state1 --> (130, 26, -150, 168, 0, 0, (0, 0)) (<class 'tuple'>)


In [1507]:
# Scoring and get the cost
%f p1 -->
%f ( 觸地了嗎？ ) p1 100 <= -->
%f s1 -->
%f ( 安全著陸嗎？ ) s1 abs 40 <= -->
%f ( 若安全著陸，所剩油料就是分數 ) fuel1 -->
%f ( 觸地時的 power ) powr0 -->

p1 --> 26.200499999998563 (<class 'float'>)
( 觸地了嗎？ ) p1 100 <= --> True (<class 'bool'>)
s1 --> -150.41700000000003 (<class 'float'>)
( 安全著陸嗎？ ) s1 abs 40 <= --> False (<class 'bool'>)
( 若安全著陸，所剩油料就是分數 ) fuel1 --> 168 (<class 'int'>)
( 觸地時的 power ) powr0 --> 0 (<class 'int'>)


In [1910]:
# 墜毀時的速度減到+-40 以內需要多少油料？ 從當時的 power 開始要用力噴火減速。
# 結果發現，修正自由落體墜毀需要的油料繼續從 fuel 裡扣除，
# 最後修正到 vspeed = -40 時 fuel = -866 此即這個 genome 的分數了。
%f ( 前 ) state1 -->
step += 1
a1 = 0
powr1 = min(powr0 + 1, 4)
fuel1 = fuel0 - powr1
force = g + powr1
s1 = s0 + force
p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))
p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
%f ( 後 ) state1 -->

# 最終把 vspeed 調整回到 -40 所剩油料是 -1434
# ( 前 ) state1 --> (531, -39454, -41, -1430, 0, 4, (0, 4)) (<class 'tuple'>)
# ( 後 ) state1 --> (532, -39495, -40, -1434, 0, 4, (0, 4)) (<class 'tuple'>)

( 前 ) state1 --> (531, -39454, -41, -1430, 0, 4, (0, 4)) (<class 'tuple'>)
( 後 ) state1 --> (532, -39495, -40, -1434, 0, 4, (0, 4)) (<class 'tuple'>)


# 筆記


2021.4.24 05:43 Scoring 要有一套客觀的計算方式. I have a much better idea now, finally. See [OneNote2021 >> Mars Lander - Episode 1](https://onedrive.live.com/view.aspx?resid=A796EA18AC8C1DA9%2115651&id=documents&wd=target%28Genetic%20Algorithm.one%7C3D994361-3562-4730-99E4-DAC83DAD73F2%2FMars%20Lander%20-%20Episode%201%7C156EADBB-D6A0-4C92-8BDD-043C48CC8271%2F%29)
# 寫出評分 function 

- 到達地表時的 位置、速度、角度、油料 決定得分。
- 與平地的距離要越近越好。

`[X]` 點到線的距離 `distance = np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1)` where p2-p1 is the line and p3 is the point. 

`[ ]` 判斷著地的方法就很簡單了，用 ship 的位置來算與每個 surface 的距離，如果到了平地就是降落了，如果到了其他 surface 就是撞毀了。

`[ ]` 降落平地還要看角度(0)、速度(20,40)，不對還是失敗。

`[ ]` 降落平地之後，油料用得少決定勝負。

1. Crashed 或 landed 得分都是與 flate ground 的直線距離 (越近越好) 
   與 B0 = np.linalg.norm(v(7000,3000)) 的差距，最高分就是這個值。

2. landed 還有加分 bonus, 角度越接近 0 越好，Bonus 要比前一個分數高，使得 crash 完全沒得比。
   角度是 -90~90 之間，故 +-90度得 B0 分，而 0度得 2B0 分，推導出: <br>
    `bonus_angle(an) = 2B0 - (B0/90)*abs(an)`

3. 水平速度要 20 以下，垂直速度要 40 以下。同上，水平速度 500 時得 B0 分，而 20 以下時得 2B0 分<br>
    `bonus_sh(sh) = 2B0 - (B0/500)*abs(0 if sh<20 else sh)`<br>
    `bonus_sv(sv) = 2B0 - (B0/500)*abs(0 if sv<40 else sv)`

4. 如果三個都滿足，再加上 fuel 看剩多少。都用完了得 B0 分，都沒用得 2B0 分。

    `bonus_fuel(fuel) = B0 + (B0/Fuel)*(fuel)`


### 做出一個 class 叫作 Ship 
有油料、位置、速度、角度、引擎出力、軌跡、從開始到落地的一長串 chromosome 也就是一長串的基因 genes (rotate,power) 、還有 score。

先亂撒一組命令 > get the distance if alives then repeat else if crash then record the final distance (to the flate ground) as its score. 嘗試的過程不評分，要撞毀了或落地了才評分。
如果 parent 都撞毀了，誰好? 簡化地形為 surface 一律朝上，則越接近 flate ground 的就是越好。若有凹洞, 也就是 surface 有朝下時，要另外想辦法。如果 parent 都撞毀了，但是其子代 gene 都用完了還 inTheAir 就要自己用亂數探索自己的前途，最終要撞毀了或落地了才評分這個子代。經過 crossover and mutation 或單純的 exploration 都一定會跑出雜枝，不影響評分與演化過程因為會被 ship.move() 修正好。


## 用 Genetic Algorithm 先找出一條成功的路徑

一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。<br>

In [ ]:
# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        p0 = v(2500, 2700)  # Initial position
        a0 = 0        # absolute angle of the ship 
        pwr0 = 0      # 引擎出力
        s0 = v(0,0)     # Initial speed vector
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break

In [ ]:
epoch = 0 

In [ ]:
# mating 用 parents 排列組合出下一代
spouses = [c for c in combinations(parents, 2)]

In [ ]:
# crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
offspring = []
for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
    offspring_crossover = crossover(spouses[i])
    if len(offspring_crossover) >= epoch :
        # 不一定 mutate 得到，看長度夠不夠
        offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
    offspring.append([0,offspring_crossover])  # _working_

In [ ]:
# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 手動連續跌代 (舊版，全部 gene 一起 mutate)

### 手動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
# mating 用 parents 排列組合出成對的配偶
spouses = [c for c in combinations(parents, 2)]

# crossover 產生下一代，隨即令其突變
offspring = []
for i in range(len(spouses)):
    offspring.append([0,list(map(mutation,crossover(spouses[i])))])

# evaluate the offspring populataion
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
for i in range(len(offspring)):
    offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200
    ship = Ship(p0, s0, a0, pwr0, fuel0)
    ship.evaluate(offspring[i][1])
    offspring[i][0] = ship.score
    %f i ship :> score -2 slice -->
epoch += 1 
# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 自動連續跌代

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

In [ ]:
%run mars.py

# Get population 
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        p0 = v(2500, 2700)  # Initial position
        a0 = 0        # absolute angle of the ship 
        pwr0 = 0      # 引擎出力
        s0 = v(0,0)     # Initial speed vector
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
# 開始自動演化
high0 = 0
for j in range(1000):
    # mating 用 parents 排列組合出成對的配偶
    spouses = [c for c in combinations(parents, 2)]

    # crossover 產生下一代，隨即令其突變
    offspring = []
    for i in range(len(spouses)):
        offspring.append([0,list(map(mutation,crossover(spouses[i])))])

    # evaluate the offspring populataion
    p0 = v(2500, 2700)  # Initial position
    a0 = 0        # absolute angle of the ship 
    pwr0 = 0      # 引擎出力
    s0 = v(0,0)     # Initial speed vector
    for i in range(len(offspring)):
        offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.evaluate(offspring[i][1])
        offspring[i][0] = ship.score

    # 上下兩代組合成新一輪的 parents 
    parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
    high1 = [row[0] for row in parents][4]
    if j%10 == 0 : print(f"#{j}", end=" ") 
    if high0 != high1 : print(f"{j}:{high1}")
    high0 = high1

# evaluate a specific chromosome
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(parents[4][1])
%f ship :> make_score() ship :> score -->
%f ship :> chromosome -->


# 修正 mutation 的方式
_working_
2019/06/13 17:41 Mutation 的方式不好，咱目前是一次所有的 gene 全部 mutate，可能是造成訓練效果不佳的原因！因為某個 gene 的 mutation 再怎麼好效果被沖淡了，無法彰顯。所以要一組一組從頭照順序來讓 Mutation 找出最佳組合。這樣看來，窮舉是 93 種組合 for 一組 gene (-15~15, -1~1)，我目前是 10 個 offspring 只有 1/9, 將來考慮改多點。窮舉可能不見得好，因為它假設後面的一長串都是對的才有意義，當然這大有疑問，所以要大家一起慢慢演化才好。從頭 mutation 到尾 (每條 chromosome 的長度不一樣，看哪一條最長都要涵蓋到) 就是一輪 epoch. 

### 手動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

In [ ]:
%run mars.py

In [ ]:
epoch = 0 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector

In [ ]:
# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break

In [ ]:
# mating 用 parents 交配出親代的排列組合
spouses = [c for c in combinations(parents, 2)]

# crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
offspring = []
for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
    offspring_crossover = crossover(spouses[i])
    if len(offspring_crossover) >= epoch :
        # 不一定 mutate 得到，看長度夠不夠
        offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
    offspring.append([0,offspring_crossover])

offspring_length = len(offspring)
epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
for i in range(offspring_length):
    offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
    ship = Ship(p0, s0, a0, pwr0, fuel0)
    chromosome_length = ship.evaluate(offspring[i][1])
    if epoch > chromosome_length : epoch_end += 1
    offspring[i][0] = ship.score
    %f i ship :> score -2 slice -->
        
if epoch_end >= offspring_length :
    %f ." End of epoch"
    epoch = 0
else:
    epoch += 1
    %f epoch --> 



# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 自動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop
%run mars.py

# constants of the game world 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector

# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
high0 = 0            
for cycle in range(1000):
    epoch = 0  # a new cycle 
    while(True):
        # mating 用 parents 交配出親代的排列組合
        spouses = [c for c in combinations(parents, 2)]

        # crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
        offspring = []
        for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
            offspring_crossover = crossover(spouses[i])
            if len(offspring_crossover) >= epoch :
                # 不一定 mutate 得到，看長度夠不夠
                offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
            offspring.append([0,offspring_crossover])

        # 為這些子代評分
        offspring_length = len(offspring)
        epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
        for i in range(offspring_length):
            offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
            ship = Ship(p0, s0, a0, pwr0, fuel0)
            chromosome_length = ship.evaluate(offspring[i][1])
            if epoch > chromosome_length : epoch_end += 1
            offspring[i][0] = ship.score
                
        # 上下兩代組合成新一輪的 parents 
        parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
        high1 = parents[4][0]
        if high0 != high1 :
            print(f"cycle:{cycle} epoch:{epoch} highest score:{high1}") 
            high0 = high1

        if epoch_end >= offspring_length :
            print(f"cycle:{cycle} ~~~~ End of epoch ~~~~")
            break
        else:
            epoch += 1

In [ ]:
parents[4][0]

# evaluate a specific chromosome

In [6]:
# evaluate a specific chromosome
chromosome = [[-15, 0], [0, 1], [-1, 2], [6, 1], [20, 0], [5, 1], [17, 0], [2, 1], [-12, 2], [-1, 3], [14, 4], [8, 4], [-3, 4], [-14, 3], [-26, 3], [-11, 3], [0, 4], [4, 3], [-11, 3], [-5, 4], [10, 3], [8, 2], [-7, 1], [-14, 2], [-20, 3], [-22, 4], [-7, 3], [-17, 4], [-17, 4], [-14, 3], [-29, 4], [-44, 4], [-35, 3], [-20, 4], [-35, 4], [-48, 4], [-46, 4], [-42, 4], [-27, 4], [-15, 4], [-19, 3], [-22, 4], [-25, 4], [-25, 3], [-19, 3], [-26, 3], [-24, 4], [-34, 4], [-49, 4], [-34, 3], [-49, 4], [-34, 3], [-49, 4], [-42, 4], [-33, 4], [-41, 3], [-42, 3], [-41, 2], [-26, 3], [-11, 2], [4, 1], [-11, 0], [-1, 1], [-15, 0], [0, 1],[0,0]] 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(chromosome)
%f ship :> score -->
%f ship :> chromosome -->
%f ship :> inTheAir -->
%f ship :> p0 -->     
trace = list(map(lambda x:np.round(x,0), ship.trace))
%f trace -->
%f ship :> trace -->



ship :> score --> 63531.77246565923 (<class 'numpy.float64'>)
ship :> chromosome --> [[-15, 0], [0, 1], [-1, 2], [6, 1], [20, 0], [5, 1], [17, 0], [2, 1], [-12, 2], [-1, 3], [14, 4], [8, 4], [-3, 4], [-14, 3], [-26, 3], [-11, 3], [0, 4], [4, 3], [-11, 3], [-5, 4], [10, 3], [8, 2], [-7, 1], [-14, 2], [-20, 3], [-22, 4], [-7, 3], [-17, 4], [-17, 4], [-14, 3], [-29, 4], [-44, 4], [-35, 3], [-20, 4], [-35, 4], [-48, 4], [-46, 4], [-42, 4], [-27, 4], [-15, 4], [-19, 3], [-22, 4], [-25, 4], [-25, 3], [-19, 3], [-26, 3], [-24, 4], [-34, 4], [-49, 4], [-34, 3], [-49, 4], [-34, 3], [-49, 4], [-42, 4], [-33, 4], [-41, 3], [-42, 3], [-41, 2], [-26, 3], [-11, 2], [4, 1], [-11, 0], [-1, 1], [-15, 0], [0, 1], [0, 0]] (<class 'list'>)
ship :> inTheAir --> False (<class 'numpy.bool_'>)
ship :> p0 --> [4066.25684777   69.90764574] (<class 'numpy.ndarray'>)
trace --> [array([2500., 2698.]), array([2500., 2693.]), array([2500., 2686.]), array([2500., 2676.]), array([2500., 2664.]), array([2500., 2648.]),

In [4]:
%debug

> c:\users\hcche\onedrive\文件\jupyter notebooks\codingame\mars lander\mars.py(119)make_score()
    117                         # 這裡要 大於/等於 0 而不是只 大於 0 ，這純粹是看 CodinGame 怎麼寫法。
    118         self.score = B0 - distances[flate]  # distance to target
--> 119         assert self.p0[1] > 200, "close to the problem point"
    120         if not self.inTheAir:
    121             if shortest != flate : self.score = -1  # 撞毀了

ipdb> self.inTheAir
True
ipdb> shortest
4
ipdb> flate
4
ipdb> exit


In [5]:
%f log :> [0] (see)


{
    "step": 0,
    "length": 91,
    "x": 2500,
    "y": 2500,
    "hspeed": 0,
    "vspeed": 0,
    "fuel": 500,
    "angle": 0,
    "power": 0,
    "cmd": [
        -99,
        -99
    ]
}


In [7]:
gene = log[0]
gene['step']
%f gene :> ['vspeed'] -->

gene :> ['vspeed'] --> 0 (<class 'int'>)


In [28]:
gene = log[0]
%f gene -->

gene --> {'step': 0, 'length': 36, 'x': 2500, 'y': 2500, 'hspeed': 0, 'vspeed': 0, 'fuel': 500, 'angle': 0, 'power': 0, 'cmd': (-99, -99)} (<class 'dict'>)


In [2]:
log

[{'step': 0,
  'length': 91,
  'x': 2500,
  'y': 2500,
  'hspeed': 0,
  'vspeed': 0,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (-99, -99)},
 {'step': 1,
  'length': 91,
  'x': 2500,
  'y': 2498,
  'hspeed': 0,
  'vspeed': -4,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 2,
  'length': 91,
  'x': 2500,
  'y': 2493,
  'hspeed': 0,
  'vspeed': -7,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 3,
  'length': 91,
  'x': 2500,
  'y': 2483,
  'hspeed': 0,
  'vspeed': -11,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 4,
  'length': 91,
  'x': 2500,
  'y': 2470,
  'hspeed': 0,
  'vspeed': -15,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 5,
  'length': 91,
  'x': 2500,
  'y': 2454,
  'hspeed': 0,
  'vspeed': -19,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 6,
  'length': 91,
  'x': 2500,
  'y': 2433,
  'hspeed': 0,
  'vspeed': -22,
  'fuel': 500,
  'angle':

In [97]:
np.array([1,1])

array([1, 1])

In [98]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [99]:
np.zeros((4, 1))

array([[0.],
       [0.],
       [0.],
       [0.]])

In [100]:
# creating a 1-D list (Horizontal)
list1 = [1, 2, 3]
  
# creating a 1-D list (Vertical)
list2 = [[10],
        [20],
        [30]]
  
# creating a vector1
# vector as row
vector1 = np.array(list1)
  
# creating a vector 2
# vector as column
vector2 = np.array(list2)
  
  
# showing horizontal vector
print("Horizontal Vector")
print(vector1)
  
print("----------------")
  
# showing vertical vector
print("Vertical Vector")
print(vector2)

Horizontal Vector
[1 2 3]
----------------
Vertical Vector
[[10]
 [20]
 [30]]


In [ ]:
NumPy is a general-purpose array-processing package. It provides a high-performance multidimensional array object, and tools for working with these arrays. It is the fundamental package for scientific computing with Python. Numpy is basically used for creating array of n dimensions. 

Vector are built from components, which are ordinary numbers. We can think of a vector as a list of numbers, and vector algebra as operations performed on the numbers in the list. In other words vector is the numpy 1-D array. 

In order to create a vector we use np.array method. 

Syntax : np.array(list) 

Argument : It take 1-D list it can be 1 row and n columns or n rows and 1 column 

 
 

 
 

Return : It returns vector which is numpy.ndarray 

Note : We can create vector with other method as well which return 1-D numpy array for example np.arange(10), np.zeros((4, 1)) gives 1-D array, but most appropriate way is using np.array with the 1-D list. 

Creating a Vector 
In this example we will create a horizontal vector and a vertical vector 

 
 

 
 
 

# importing numpy 

import numpy as np 

   

# creating a 1-D list (Horizontal) 

list1 = [1, 2, 3] 

   

# creating a 1-D list (Vertical) 

list2 = [[10], 

        [20], 

        [30]] 

   

# creating a vector1 

# vector as row 

vector1 = np.array(list1) 

   

# creating a vector 2 

# vector as column 

vector2 = np.array(list2) 

   

   

# showing horizontal vector 

print("Horizontal Vector") 

print(vector1) 

   

print("----------------") 

   

# showing vertical vector 

print("Vertical Vector") 

print(vector2) 

Output : 

Horizontal Vector 
[1 2 3] 
---------------- 
Vertical Vector 
[[10] 
[20] 
[30]] 
 

Basic Arithmetic operation: 
In this example we will see do arithmetic operations which are element-wise between two vectors of equal length to result in a new vector with the same length 

 
 

 
 
 

# importing numpy 

import numpy as np 

   

# creating a 1-D list (Horizontal) 

list1 = [5, 6, 9] 

   

# creating a 1-D list (Horizontal) 

list2 = [1, 2, 3] 

   

# creating first vector  

vector1 = np.array(list1) 

   

# printing vector1 

print("First Vector          : " + str(vector1)) 

   

# creating secodn vector 

vector2 = np.array(list2) 

   

# printing vector2 

print("Second Vector         : " + str(vector2)) 

   

# adding both the vector 

# a + b = (a1 + b1, a2 + b2, a3 + b3) 

addition = vector1 + vector2 

   

# printing addition vector 

print("Vector Addition       : " + str(addition)) 

   

# subtracting both the vector 

# a - b = (a1 - b1, a2 - b2, a3 - b3) 

subtraction = vector1 - vector2 

   

# printing addition vector 

print("Vector Substraction   : " + str(subtraction)) 

   

# multiplying  both the vector 

# a * b = (a1 * b1, a2 * b2, a3 * b3) 

multiplication = vector1 * vector2 

   

# printing multiplication vector 

print("Vector Multiplication : " + str(multiplication)) 

   

# dividing  both the vector 

# a / b = (a1 / b1, a2 / b2, a3 / b3) 

division = vector1 / vector2 

   

# printing multiplication vector 

print("Vector Division       : " + str(multiplication)) 

    

Output : 

First Vector          : [5 6 9] 
Second Vector         : [1 2 3] 
Vector Addition       : [ 6  8 12] 
Vector Substraction   : [4 4 6] 
Vector Multiplication : [ 5 12 27] 
Vector Division       : [ 5 12 27] 
 

Vector Dot Product 
In mathematics, the dot product or scalar product is an algebraic operation that takes two equal-length sequences of numbers and returns a single number. 
For this we will use dot method. 

 
 

 
 
 

# importing numpy 

import numpy as np 

   

# creating a 1-D list (Horizontal) 

list1 = [5, 6, 9] 

   

# creating a 1-D list (Horizontal) 

list2 = [1, 2, 3] 

   

# creating first vector  

vector1 = np.array(list1) 

   

# printing vector1 

print("First Vector  : " + str(vector1)) 

   

# creating secodn vector 

vector2 = np.array(list2) 

   

# printing vector2 

print("Second Vector : " + str(vector2)) 

   

# getting dot product of both the vectors 

# a . b = (a1 * b1 + a2 * b2 + a3 * b3) 

# a . b = (a1b1 + a2b2 + a3b3) 

dot_product = vector1.dot(vector2) 

   

# printing dot product 

print("Dot Product   : " + str(dot_product)) 

First Vector  : [5 6 9] 
Second Vector : [1 2 3] 
Dot Product   : 44 
 

Vector-Scalar Multiplication 
Multiplying a vector by a scalar is called scalar multiplication. To perform scalar multiplication, we need to multiply the scalar by each component of the vector. 

 
 

 
 
 

# importing numpy 

import numpy as np 

   

# creating a 1-D list (Horizontal) 

list1 = [1, 2, 3] 

   

# creating first vector  

vector = np.array(list1) 

   

# printing vector1 

print("Vector  : " + str(vector)) 

   

# scalar value  

scalar = 2 

   

# printing scalar value 

print("Scalar  : " + str(scalar)) 

    

# getting scalar multiplication value 

# s * v = (s * v1, s * v2, s * v3) 

scalar_mul = vector * scalar 

   

# printing dot product 

print("Scalar Multiplication : " + str(scalar_mul)) 

   

   

    

Output 

Vector  : [1 2 3] 
Scalar  : 2 
Scalar Multiplication : [2 4 6] 
 
 

Attention geek! Strengthen your foundations with the Python Programming Foundation Course and learn the basics. 

To begin with, your interview preparations Enhance your Data Structures concepts with the Python DS Course. 

My Personal Notes arrow_drop_up 